<a href="https://colab.research.google.com/github/sathu0622/25-26J-438-AI-Powered-LMS-for-Visually-Impaired-Students/blob/AI-Powered-Braille-to-Text-Conversion-and-Automated-Evaluation-System-for-O%2FL-History-Examinations/meta_llama_Meta_Llama_3_8B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  STEP 1 — Install Dependencies
# ============================================================
!pip install -q transformers accelerate bitsandbytes peft datasets sentencepiece openpyxl scikit-learn

# ============================================================
#  STEP 2 — Mount Google Drive & Clear Memory
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# Clear any cached memory
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
print("✅ Memory cleared")

# ============================================================
#  STEP 3 — Import Libraries
# ============================================================
import pandas as pd
import json
import torch
import numpy as np
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
from sklearn.model_selection import train_test_split
from huggingface_hub import login

print("✅ All libraries imported successfully")

# ============================================================
#  STEP 3.5 — Hugging Face Authentication
# ============================================================
print("\n" + "="*60)
print("🔐 HUGGING FACE AUTHENTICATION REQUIRED")
print("="*60)
print("Llama 3.1 is a gated model. You need to:")
print("1. Go to: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct")
print("2. Click 'Request Access' and accept the terms")
print("3. Create a token at: https://huggingface.co/settings/tokens")
print("4. Enter your token below")
print("="*60 + "\n")

# Get token from user
from getpass import getpass
hf_token = getpass("Enter your Hugging Face token (input will be hidden): ")

# Login to Hugging Face
try:
    login(token=hf_token, add_to_git_credential=True)
    print("✅ Successfully authenticated with Hugging Face!")
except Exception as e:
    print(f"❌ Authentication failed: {str(e)}")
    print("\nPlease make sure:")
    print("1. You've requested access to Llama 3.1 model")
    print("2. Your access has been approved (check your email)")
    print("3. Your token has 'read' permissions")
    raise

# ============================================================
#  STEP 4 — Load Excel Dataset from Google Drive
# ============================================================
dataset_path = "/content/drive/MyDrive/Model/Final.xlsx"

try:
    df = pd.read_excel(dataset_path)

    # Check and standardize column names
    df.columns = df.columns.str.strip()  # Remove any whitespace

    # Handle different possible column names
    column_mapping = {}
    for col in df.columns:
        col_lower = col.lower()
        if 'question' in col_lower:
            column_mapping[col] = 'question'
        elif 'answer' in col_lower:
            column_mapping[col] = 'answer'

    df = df.rename(columns=column_mapping)

    # Verify required columns exist
    if 'question' not in df.columns or 'answer' not in df.columns:
        raise ValueError(f"Required columns not found. Found columns: {list(df.columns)}")

    print(f"✅ Loaded {len(df)} questions from dataset")
    print(f"✅ Columns: {list(df.columns)}")
    print("\nFirst 5 rows:")
    print(df.head())

except FileNotFoundError:
    print("❌ Error: Dataset file not found!")
    print(f"Please ensure your Excel file is at: {dataset_path}")
    print("Expected columns: 'question' and 'answer'")
except Exception as e:
    print(f"❌ Error loading dataset: {str(e)}")
    print(f"Columns found: {list(df.columns) if 'df' in locals() else 'Unable to read file'}")

# ============================================================
#  STEP 5 — Format Dataset for Llama 3.1 Training
# ============================================================
def format_instruction(question, answer):
    """Format in Llama 3.1 Instruct chat template"""
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert in Sri Lankan O/L History. Provide accurate and concise answers to history questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{answer}<|eot_id|>"""

# Create formatted training data
formatted_data = []
for _, row in df.iterrows():
    formatted_text = format_instruction(row["question"], row["answer"])
    formatted_data.append({
        "text": formatted_text
    })

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(pd.DataFrame(formatted_data))
print(f"✅ Formatted {len(train_dataset)} training examples")

# ============================================================
#  STEP 6 — Load Llama 3.1 8B Model with 4-bit Quantization
# ============================================================
model_name = "meta-llama/Llama-3.1-8B-Instruct"

print("\n🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    trust_remote_code=True
)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.padding_side = "right"
print("✅ Tokenizer loaded")

print("\n🔄 Loading model with 4-bit quantization...")

from transformers import BitsAndBytesConfig

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)
print("✅ Model loaded successfully")

# ============================================================
#  STEP 7 — Configure LoRA for Efficient Fine-tuning
# ============================================================
lora_config = LoraConfig(
    r=16,                                    # LoRA rank
    lora_alpha=32,                           # LoRA alpha scaling
    target_modules=[                         # Target attention modules
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,                       # Dropout for regularization
    bias="none",                             # Don't train biases
    task_type="CAUSAL_LM"                    # Task type
)

model = get_peft_model(model, lora_config)
print("\n✅ LoRA configuration applied")
model.print_trainable_parameters()

# ============================================================
#  STEP 8 — Tokenize Dataset
# ============================================================
def tokenize_function(examples):
    """Tokenize the text data"""
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )
    # Set labels for causal language modeling
    result["labels"] = result["input_ids"].copy()
    return result

print("\n🔄 Tokenizing dataset...")
tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing"
)
print("✅ Dataset tokenized")

# ============================================================
#  STEP 9 — Create Validation Split
# ============================================================
from sklearn.model_selection import train_test_split

# Split dataset: 80% training, 20% validation
train_indices, val_indices = train_test_split(
    range(len(tokenized_dataset)),
    test_size=0.2,
    random_state=42
)

train_subset = tokenized_dataset.select(train_indices)
val_subset = tokenized_dataset.select(val_indices)

print(f"\n✅ Dataset split:")
print(f"   Training samples: {len(train_subset)}")
print(f"   Validation samples: {len(val_subset)}")

# ============================================================
#  STEP 10 — Set Up Training Arguments
# ============================================================
output_dir = "/content/drive/MyDrive/Model/ol_history_model"

training_args = TrainingArguments(
    # Output directory
    output_dir=output_dir,

    # Training parameters
    num_train_epochs=3,                      # Number of epochs
    per_device_train_batch_size=1,           # Reduced from 2 to 1
    per_device_eval_batch_size=1,            # Reduced from 2 to 1
    gradient_accumulation_steps=8,           # Increased from 4 to 8

    # Optimizer settings
    learning_rate=2e-4,                      # Learning rate
    weight_decay=0.01,                       # Weight decay
    warmup_steps=50,                         # Warmup steps
    optim="paged_adamw_8bit",               # 8-bit optimizer

    # Evaluation settings
    eval_strategy="epoch",                   # Evaluate after each epoch
    eval_steps=None,                         # Eval every N steps (None = use strategy)
    load_best_model_at_end=True,            # Load best model at end
    metric_for_best_model="eval_loss",      # Metric to track
    eval_accumulation_steps=4,               # Accumulate eval to save memory

    # Logging and saving
    logging_steps=20,                        # Increased from 10
    save_strategy="epoch",                   # Save after each epoch
    save_total_limit=1,                      # Keep only 1 checkpoint (was 2)

    # Performance & Memory optimization
    fp16=True,                               # Mixed precision training
    gradient_checkpointing=True,             # Save memory
    max_grad_norm=0.3,                       # Gradient clipping

    # Other settings
    report_to="none",                        # Don't report to wandb/tensorboard
    remove_unused_columns=False,             # Keep all columns
    dataloader_pin_memory=False,             # Disable pin memory to save RAM
)

print("\n✅ Training arguments configured (Memory optimized)")

# ============================================================
#  STEP 11 — Define Accuracy Metrics
# ============================================================
def compute_metrics(eval_pred):
    """
    Compute perplexity and accuracy metrics for evaluation
    """
    predictions, labels = eval_pred

    # Calculate perplexity from loss
    # Perplexity = exp(loss)
    loss = np.mean(predictions)
    perplexity = np.exp(loss)

    return {
        "perplexity": perplexity,
    }

print("✅ Metrics function defined")

# ============================================================
#  STEP 12 — Initialize Trainer with Metrics
# ============================================================
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal LM, not masked LM
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=val_subset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✅ Trainer initialized with validation dataset")

# ============================================================
#  STEP 13 — Train the Model with Evaluation
# ============================================================

# Clear memory before training
import gc
gc.collect()
torch.cuda.empty_cache()
print("✅ Memory cleared before training")

print("\n" + "="*60)
print("🚀 STARTING TRAINING WITH VALIDATION")
print("="*60)
print(f"Training samples: {len(train_subset)}")
print(f"Validation samples: {len(val_subset)}")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print("="*60 + "\n")

# Start training
try:
    train_result = trainer.train()
    print("\n✅ Training completed!")
except RuntimeError as e:
    if "out of memory" in str(e):
        print("\n❌ GPU Out of Memory Error!")
        print("\n🔧 Solutions:")
        print("1. Restart runtime: Runtime → Restart runtime")
        print("2. After restart, the code will use even smaller batch size")
        print("3. Or reduce max_length in tokenization (line 189) to 256")
        raise
    else:
        raise

# ============================================================
#  STEP 14 — Display Training Results
# ============================================================
print("\n" + "="*60)
print("📊 TRAINING RESULTS")
print("="*60)

# Training metrics
metrics = train_result.metrics
print(f"Final Training Loss: {metrics.get('train_loss', 'N/A'):.4f}")
print(f"Training Runtime: {metrics.get('train_runtime', 0):.2f} seconds")
print(f"Samples per second: {metrics.get('train_samples_per_second', 0):.2f}")

# Get final evaluation metrics
print("\n🔍 Evaluating on validation set...")
eval_metrics = trainer.evaluate()

print("\n" + "="*60)
print("📈 VALIDATION RESULTS")
print("="*60)
print(f"Validation Loss: {eval_metrics.get('eval_loss', 'N/A'):.4f}")
print(f"Perplexity: {eval_metrics.get('eval_perplexity', 'N/A'):.4f}")
print(f"Validation Runtime: {eval_metrics.get('eval_runtime', 0):.2f} seconds")
print("="*60)

# Calculate improvement metrics
print("\n📉 TRAINING PROGRESS:")
if hasattr(trainer.state, 'log_history'):
    # Get first and last training loss
    train_losses = [log['loss'] for log in trainer.state.log_history if 'loss' in log]
    if len(train_losses) >= 2:
        initial_loss = train_losses[0]
        final_loss = train_losses[-1]
        improvement = ((initial_loss - final_loss) / initial_loss) * 100
        print(f"Initial Training Loss: {initial_loss:.4f}")
        print(f"Final Training Loss: {final_loss:.4f}")
        print(f"Loss Reduction: {improvement:.2f}%")

    # Get validation losses per epoch
    eval_losses = [log['eval_loss'] for log in trainer.state.log_history if 'eval_loss' in log]
    if eval_losses:
        print(f"\n📊 Validation Loss per Epoch:")
        for i, loss in enumerate(eval_losses, 1):
            print(f"   Epoch {i}: {loss:.4f}")

print("="*60)

# ============================================================
#  STEP 15 — Save the Fine-tuned Model
# ============================================================
final_model_path = f"{output_dir}/final_lora_model"

print(f"\n🔄 Saving model to {final_model_path}...")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

# Save training metrics
metrics_path = f"{output_dir}/training_metrics.json"
all_metrics = {
    "train_loss": metrics.get('train_loss'),
    "train_runtime": metrics.get('train_runtime'),
    "train_samples_per_second": metrics.get('train_samples_per_second'),
    "eval_loss": eval_metrics.get('eval_loss'),
    "eval_perplexity": eval_metrics.get('eval_perplexity'),
    "eval_runtime": eval_metrics.get('eval_runtime'),
    "num_train_samples": len(train_subset),
    "num_val_samples": len(val_subset),
    "num_epochs": training_args.num_train_epochs,
}

import json
with open(metrics_path, 'w') as f:
    json.dump(all_metrics, f, indent=2)

print(f"✅ Metrics saved to {metrics_path}")

print("\n" + "="*60)
print("✅ MODEL TRAINING COMPLETE!")
print("="*60)
print(f"Model saved at: {final_model_path}")
print(f"Training Loss: {metrics.get('train_loss', 'N/A'):.4f}")
print(f"Validation Loss: {eval_metrics.get('eval_loss', 'N/A'):.4f}")
print(f"Perplexity: {eval_metrics.get('eval_perplexity', 'N/A'):.4f}")
print("\nYou can now use this model for evaluation.")
print("Next step: Run the evaluation code to test student answers.")
print("="*60)

In [ ]:
# ============================================================
#  O/L HISTORY ANSWER EVALUATION SYSTEM - FINAL & CORRECTED
# ============================================================

# =======================
# STEP 1 — Install libs
# =======================
!pip install -q sentence-transformers transformers bitsandbytes peft accelerate huggingface_hub scikit-learn

# =======================
# STEP 2 — Mount Drive
# =======================
from google.colab import drive
drive.mount('/content/drive')

# =======================
# STEP 3 — Hugging Face Login
# =======================
from huggingface_hub import login
from getpass import getpass

hf_token = getpass("Enter Hugging Face Token: ")
login(token=hf_token, add_to_git_credential=True)

# =======================
# STEP 4 — Imports
# =======================
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer

print("✅ Libraries loaded")

# =======================
# STEP 5 — Model Paths
# =======================
BASE_MODEL = "meta-llama/Llama-3.1-8B-Instruct"
LORA_MODEL_PATH = "/content/drive/MyDrive/Model/ol_history_model/final_lora_model"

# =======================
# STEP 6 — Quantization
# =======================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# =======================
# STEP 7 — Load Model
# =======================
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(base_model, LORA_MODEL_PATH)
model.eval()

print("✅ LLaMA + LoRA model loaded")

# =======================
# STEP 8 — Semantic Model
# =======================
sbert = SentenceTransformer("all-MiniLM-L6-v2")
print("✅ SBERT loaded")

# =======================
# STEP 9 — Generate Model Answer
# =======================
def generate_correct_answer(question):
    """
    Generate a model answer for a given O/L History question.
    """

    # Set random seed for reproducibility
    torch.manual_seed(42)

    # Prepare the prompt
    prompt = f"""
You are an expert Sri Lankan O/L History teacher.
Answer the question clearly and factually in detailed, exam-style narrative suitable for a Grade 11 student.
Include all key historical points, but keep language simple.
Do NOT add unnecessary commentary or endnotes.

Question:
{question}

Answer:
"""

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,   # shorter for exam style
            temperature=0.5,      # deterministic output
            top_p=1.0,            # consider all words
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and clean the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Answer:")[-1].strip()


# =======================
# STEP 10 — Scoring Metrics
# =======================
def semantic_similarity(correct, student):
    emb1 = sbert.encode(correct, convert_to_tensor=True)
    emb2 = sbert.encode(student, convert_to_tensor=True)
    return round(float(util.cos_sim(emb1, emb2)) * 100, 2)

def keyword_overlap_score(correct, student):
    vectorizer = TfidfVectorizer(stop_words="english", max_features=20)
    try:
        tfidf = vectorizer.fit_transform([correct, student])
        features = vectorizer.get_feature_names_out()

        correct_words = set(features[i] for i,v in enumerate(tfidf[0].toarray()[0]) if v > 0)
        student_words = set(features[i] for i,v in enumerate(tfidf[1].toarray()[0]) if v > 0)

        if not correct_words:
            return 0.0

        return round(len(correct_words & student_words) / len(correct_words) * 100, 2)
    except:
        return 0.0

def jaccard_similarity(correct, student):
    def tokenize(text):
        text = re.sub(r'[^a-z\s]', '', text.lower())
        return set(text.split())

    a, b = tokenize(correct), tokenize(student)
    if not a or not b:
        return 0.0

    return round(len(a & b) / len(a | b) * 100, 2)

def length_penalty(correct, student):
    r = len(student.split()) / max(len(correct.split()), 1)
    if 0.5 <= r <= 1.5:
        return 1.0
    elif r < 0.5:
        return 0.8
    return 0.9

# =======================
# STEP 11 — Error Detection
# =======================
HISTORICAL_ERRORS = [
    "british", "factory", "industrial", "ignored agriculture",
    "little impact", "did not contribute", "traveling abroad"
]

def detect_historical_errors(answer):
    errors = sum(1 for e in HISTORICAL_ERRORS if e in answer.lower())
    if errors >= 3:
        return 0.3
    elif errors == 2:
        return 0.5
    elif errors == 1:
        return 0.8
    return 1.0

# =======================
# STEP 12 — Final Score
# =======================

def calculate_final_score(correct, student):
    semantic = semantic_similarity(correct, student)
    keyword = keyword_overlap_score(correct, student)
    jaccard = jaccard_similarity(correct, student)

    length_factor = length_penalty(correct, student)
    error_factor = detect_historical_errors(student)

    # Strict weighted score
    final = (semantic * 0.65 + keyword * 0.35) * length_factor * error_factor

    if semantic >= 60 and keyword >= 50 and error_factor == 1.0: final += 5

    return round(final, 2), semantic, keyword, jaccard, error_factor

# =======================
# STEP 13 — Feedback
# =======================
def generate_feedback(score):
    if score >= 70:
        return "Excellent answer with correct historical understanding."
    elif score >= 50:
        return "Good answer, but some important points can be improved."
    elif score >= 40:
        return "Basic understanding shown, but key facts are missing."
    else:
        return "Incorrect or weak answer. Please revise the lesson."

# =======================
# STEP 14 — Evaluation
# =======================
def evaluate_student_answer(question, student_answer):
    correct_answer = generate_correct_answer(question)

    final, semantic, keyword, jaccard, error_penalty = calculate_final_score(
        correct_answer, student_answer
    )

    status = "PASS" if final >= 60 else "NEEDS IMPROVEMENT" if final >= 50 else "FAIL"

    return {
        "Question": question,
        "Student Answer": student_answer,
        "Model Answer": correct_answer,
        "Final Score (%)": final,
        "Semantic Similarity (%)": semantic,
        "Keyword Match (%)": keyword,
        "Jaccard Similarity (%)": jaccard,
        "Error Penalty": f"{int(error_penalty*100)}%",
        "Status": status,
        "Feedback": generate_feedback(final)
    }

# =======================
# STEP 15 — Display Results
# =======================
def display_results(result):
    print("\n" + "=" * 70)
    print("📊 O/L HISTORY ANSWER EVALUATION RESULTS")
    print("=" * 70)

    print("\n📝 Question:")
    print(result["Question"])

    print("\n✍️ Student Answer:")
    print(result["Student Answer"])

    print("\n📘 Model Answer:")
    print(result["Model Answer"])

    print("\n📈 Scoring Breakdown:")
    print(f"   • Final Score         : {result['Final Score (%)']}%")
    print(f"   • Semantic Similarity : {result['Semantic Similarity (%)']}%")
    print(f"   • Keyword Match       : {result['Keyword Match (%)']}%")
    print(f"   • Jaccard Similarity  : {result['Jaccard Similarity (%)']}%")
    print(f"   • Error Penalty       : {result['Error Penalty']}")

    print("\n🏁 Status:")
    print(f"   • {result['Status']}")

    print("\n💬 Feedback:")
    print(result["Feedback"])

    print("=" * 70 + "\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter Hugging Face Token: ··········
✅ Libraries loaded


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ LLaMA + LoRA model loaded
✅ SBERT loaded


In [ ]:
# =======================
# STEP 16 — INTERACTIVE MODE
# =======================
print("\n🎓 O/L HISTORY ANSWER EVALUATION SYSTEM")

question = input("\n📚 Enter the question: ").strip()
student_answer = input("\n✍️ Enter the student's answer: ").strip()

if not question or not student_answer:
    print("❌ Question and answer cannot be empty.")
else:
    result = evaluate_student_answer(question, student_answer)
    display_results(result)



🎓 O/L HISTORY ANSWER EVALUATION SYSTEM

📚 Enter the question: Explain the contributions of King Parakramabahu I to the development of Sri Lanka.

✍️ Enter the student's answer: King Parakramabahu I was mainly known for leading wars against the British and building large factories for trade. He ignored agriculture and did not contribute to religion or education. Most of his reign was spent traveling abroad and he had little impact on the development of Sri Lanka.

📊 O/L HISTORY ANSWER EVALUATION RESULTS

📝 Question:
Explain the contributions of King Parakramabahu I to the development of Sri Lanka.

✍️ Student Answer:
King Parakramabahu I was mainly known for leading wars against the British and building large factories for trade. He ignored agriculture and did not contribute to religion or education. Most of his reign was spent traveling abroad and he had little impact on the development of Sri Lanka.

📘 Model Answer:
King Parakramabahu I ruled Anuradhapura during a critical period whe